# Why pandas?

The `🐼.DataFrame` and `🐼.Series` `object`s provide [fluent interfaces](https://en.wikipedia.org/wiki/Fluent_interface) to compose complex actions that transform, visualize, and model data by [chaining](https://tomaugspurger.github.io/method-chaining).  

[![image](https://user-images.githubusercontent.com/4236275/46419595-3bac3580-c6fc-11e8-85c9-3ffe212c0021.png)](https://trestletech.com/wp-content/uploads/2015/07/dplyr.pdf#page=6)


To me, and some others, fluent interfaces are more readable than nested operations. 

[![image](https://user-images.githubusercontent.com/4236275/46419770-980f5500-c6fc-11e8-9cbd-a4ca0b830b6e.png)](https://tomaugspurger.github.io/method-chaining)

# ✋✋✋✋✋✋✋✋

I'm [Tony Fast]().ipynb_checkpoints/  I organize [PyData Atlanta]() and the [Jupyter User Group]().  I am currently interested in composing human readable programs
using notebooks.

I'm going to tell you a little bit about myself in data.  Then we will dissect my approach.

In [3]:
    from .statements import get as request
    import typing as t
    import pandas, asyncio, ibis.sql.sqlite
    from toolz.curried import *
    %matplotlib inline
    import datetime

    from .statements import get as request
    import typing as t
    import pandas, asyncio, ibis.sql.sqlite
    from toolz.curried import *
    %matplotlib inline
    import datetime

C:\Anaconda3\lib\site-packages\ibis\sql\postgres\compiler.py:223: UserWarning: locale specific date formats (%c, %x, %X) are not yet implemented for Windows
  'for %s' % platform.system()


# Accessing data.

Pandas has a bunch of read methods that you should fully take advantage of.

In [4]:
    pipe(pandas, dir, filter(flip(str.startswith)('read_')), list, pandas.DataFrame).T

    pipe(pandas, dir, filter(flip(str.startswith)('read_')), list, pandas.DataFrame).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,read_clipboard,read_csv,read_excel,read_feather,read_fwf,read_gbq,read_hdf,read_html,read_json,read_msgpack,read_parquet,read_pickle,read_sas,read_sql,read_sql_query,read_sql_table,read_stata,read_table


In [ ]:
    frames = pandas.read_html("https://en.wikipedia.org/wiki/List_of_physical_quantities")
    f"There are __{len(frames)}__ tables in this webpage."

In [ ]:
    https://chrisalbon.com/python/data_wrangling/pandas_list_comprehension/

## Caching requests

Caching our requests assures that our data remains local and we can work with out the internet.

In [ ]:
    import requests_cache; requests_cache.install_cache('idiomatic_pandas')

In [ ]:
    request??

> Sometimes I use Python type annotations to indicate the input and output values of the functions.

In [ ]:
    username = 'tonyfast'

In [ ]:
    info = await request(f"https://api.github.com/users/%s"%username); info.to_frame()

In [ ]:
    f"![]({info.avatar_url})"

## Some utitlity statements.

Accessing historical data in github requires pagination. https://developer.github.com/v3/guides/traversing-with-pagination/

`paginate` appends `"?page=%i"` to the end of our Github API requests.

In [ ]:
    def paginate(url: str, slice: slice=slice(1,3)) -> t.Generator:
        return (f"{url}?page={page}" for page in range(slice.start, slice.stop+1))

I don't have `datetime` for composing timestamps so I use `pandas.to_datetime`

In [ ]:
    def timify(df):
        at = [str for str in df.columns if str.endswith('_at')]
        df[at] = df[at].apply(pandas.to_datetime)        
        return df

`pandas.to_timedelta` helps with Δt.

In [ ]:
    pandas.to_timedelta('1 days 10 seconds')

and the difference between timestamps returns a time delta.

In [ ]:
    Δt = pandas.Series([info.created_at, info.updated_at]).pipe(pandas.to_datetime).diff().dropna(); Δt

## Let's talk about me

In [ ]:
    repos = pandas.concat(
        await asyncio.gather(*map(
            request, paginate(info.repos_url, slice(1, info.public_repos//30+1))))
    ).set_index('id').pipe(timify)
    f"""There have __{len(repos)}__ repositories associated with [__@{info.login}'s__]({info.html_url}) account after __>{Δt[1].days//365}__ years."""

In [ ]:
    events = pandas.concat(
        await asyncio.gather(*map(
            request, paginate(info.events_url.format(**{'/privacy': ''}), slice(1, 10))))
    ).set_index('id').pipe(timify)

>  __🐼 + `async` + Caching = 💪__

In [ ]:
    gists = pandas.concat(
        await asyncio.gather(*map(
            request, paginate(info.gists_url.format(**{'/gist_id': ''}), slice(1, info.public_gists//30+1))
        ))
    ).set_index('id').pipe(timify)
    f"""I've made __{len(gists)}__ gists during that time."""

In [ ]:
    stars = pandas.concat(
        await asyncio.gather(*map(
            partial(request, headers={
                'Accept': "application/vnd.github.v3.star+json"
            }), paginate(
                info.starred_url.format(**{'/owner': '', '/repo': ''}), slice(1, 700//30+1))
        ))
    ).pipe(
        lambda df: pandas.concat([df.repo.apply(pandas.Series), df.starred_at], axis=1)
    ).pipe(timify).set_index('id')
    f"""I've starred __{len(stars)}__ repositories, too."""

In [ ]:
    pandas.Index.to_series, pandas.Index.to_frame, pandas.Index.to_native_types

In [ ]:
    events.type.value_counts().plot.pie()
    f"""A look at __{username}'s__ last __300__ events.'"""

In [ ]:
    from matplotlib_venn import *; from toolz.curried.operator import *

In [ ]:
    pipe((repos, gists, stars), map(attrgetter('columns')), map(set), list, 
         partial(venn3_unweighted, set_labels=('repos', 'gists', 'stars')))

    intersections = set(repos.columns).intersection(gists.columns).intersection(stars.columns)
    f"The __{len(intersections)}__ overlapping columns are: __{', '.join(intersections)}__"

## Aggregations

`pandas.DataFrame.groupby` & `pandas.DataFrame.rolling` aggreations

In [ ]:
    monthly_actions = repos.set_index('created_at')[[]].join(
        gists.set_index('created_at')[[]]
    ).join(
        stars.set_index('created_at')[[]]
    ).sort_index().index.to_frame().groupby(
        pandas.Grouper(freq='M')
    ).count()['created_at'].rename('created')

In [ ]:
    ax = monthly_actions.plot(title='Events per month', legend=True)
    monthly_actions.rolling(3, center=True).mean().rename('rolling average').fillna(0).plot(ax=ax, legend=True, grid=True)

In [ ]:
    this_years_actions = monthly_actions[monthly_actions.index > pandas.to_datetime('2018-04-01')]
    ax = this_years_actions.plot(title='Events per month', legend=True, figsize=(24, 8), grid=True)
    this_years_actions.rolling(3, center=True).mean().rename('rolling average').fillna(0).plot(ax=ax, legend=True)
    events.set_index('created_at').index.to_frame().groupby(
        pandas.Grouper(freq='1D')
    ).count()['created_at'].rename('minor events').plot(ax=ax, legend=True, grid=True)

In [ ]:
    gist = gists.files.apply(pandas.Series).stack(level=0).apply(pandas.Series).join(gists).reset_index(-1, drop=True)

In [ ]:
    import matplotlib

Method chaingm

In [ ]:
    languages = gist.set_index('created_at').language.groupby(pandas.Grouper(freq='1Y')).value_counts().unstack().pipe(
        lambda df: languages.set_index(df.index.year))
    subplots = pandas.Series(matplotlib.pyplot.subplots(1, len(languages), figsize=(24, 5))[1], languages.index)
    
    languages.apply(
        lambda s: s.dropna().plot.bar(title=s.name, ax=subplots.loc[s.name], grid=True), axis=1)
    matplotlib.pyplot.show()
    gist.language.value_counts().plot.bar(title='total', grid=True, figsize=(24, 5));

    from . import readme as ˣ

In [ ]:
    import pandas; "Explicit is better than implicit.";

In [ ]:
    import os; os.environ.get('gh'); params = {'access_token': os.environ.get('gh')}

In [ ]:
    gh = f"https://api.github.com/users/%s"

In [ ]:
    info = pandas.read_json(gh%'tonyfast', typ=pandas.Series)

In [ ]:
    import ast
    import nbconvert

In [ ]:
    attributes = pandas.Series()
    class FindAttributes(ast.NodeVisitor):
        def visit_Attribute(self, node):   self.attributes[-1].append(node.attr) or super().generic_visit(node)
                        
        def visit_Name(self, node):  
            value = globals().get(node.id, None)
            self.attributes[-1].append(
                type(value).__name__ if isinstance(value, (pandas.DataFrame, pandas.Series)) else node.id
            ) or super().generic_visit(node)

        def __call__(self, str):
            self.attributes = list()
            for body in ast.parse(str).body:
                self.attributes.append([])
                self.visit(body)
                self.attributes[-1] = ', '.join(reversed(self.attributes[-1]))
            return pandas.Series(self.attributes).pipe(lambda s: s[s.apply(bool)])
    actions = FindAttributes()(nbconvert.get_exporter('python')().from_filename(filename='intro.ipynb')[0].replace('await ', ''))

A back of the envelope use of attributes.

In [ ]:
    actions.str.split(
        ',', expand=True
    ).stack().str.strip().value_counts().T

In [ ]:
    pandas.DataFrame.applymap, pandas.DataFrame.apply, pandas.DataFrame.pipe
    pandas.Series.apply, pandas.Series.pipe
    pandas.Index.map, pandas.Index.to_series, pandas.Index.to_frame

In [ ]:
    df = pandas.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                        'B': {0: 1, 1: 3, 2: 5},
                        'C': {0: 2, 1: 4, 2: 6}})
    pandas.melt(df, id_vars=['A'], value_vars=['B'])

In [371]:
    df.set_index('A').unstack()

    df.set_index('A').unstack()

   A
B  a    1
   b    3
   c    5
C  a    2
   b    4
   c    6
dtype: int64